<a href="https://colab.research.google.com/github/Chaross/Plascan.AI/blob/main/Plascan_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import dependencies**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tkinter import filedialog
from tkinter import Tk

**Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = "/content/drive/MyDrive/Colab Notebooks/Plascan AI/cell_images"
categories = ["Parasitized", "Uninfected"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load Data**

In [ ]:
valid_extensions = ['.jpg', '.jpeg', '.png']
data = []
for category in categories:
    path = os.path.join(data_dir, category)
    label = categories.index(category)

    for img_file in tqdm(os.listdir(path), desc=f"Loading {category}"):
        # Ignore hidden files or non-images
        if not any(img_file.lower().endswith(ext) for ext in valid_extensions):
            continue

        try:
            img_path = os.path.join(path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Couldn't load {img_path}")
                continue
            img = cv2.resize(img, (64, 64))
            data.append([img, label])
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

Loading Uninfected: 100%|██████████| 12003/12003 [10:56<00:00, 18.29it/s] 


Resize Images to 64x64

In [ ]:
img_size = 64  # Resize all images to 64x64
data = []
for category in categories:
    path = os.path.join(data_dir, category)
    label = categories.index(category)  # 0 or 1
    for img_file in tqdm(os.listdir(path), desc=f"Loading {category}"):
        try:
            img_path = os.path.join(path, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            data.append([img, label])
        except:
            continue

Loading Uninfected: 100%|██████████| 12003/12003 [02:59<00:00, 66.88it/s] 


**Shuffle and split the Data**

In [ ]:
import random
random.shuffle(data)

X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X) / 255.0  # Normalize pixel values to [0, 1]
y = np.array(y)

**Split the Data to Training and Test Set**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Build the CNN Model**

In [ ]:
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Train the Model**

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # or 'categorical_crossentropy' depending on your labels
    metrics=['accuracy']
)

# Then train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 245s 367ms/step - accuracy: 0.8102 - loss: 0.5429 - val_accuracy: 0.8187 - val_loss: 0.4842
Epoch 2/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 259s 364ms/step - accuracy: 0.9487 - loss: 0.1546 - val_accuracy: 0.9491 - val_loss: 0.1819
Epoch 3/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 223s 345ms/step - accuracy: 0.9514 - loss: 0.1512 - val_accuracy: 0.9483 - val_loss: 0.1457
Epoch 4/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 228s 353ms/step - accuracy: 0.9540 - loss: 0.1360 - val_accuracy: 0.6178 - val_loss: 0.5284
Epoch 5/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 267s 361ms/step - accuracy: 0.9595 - loss: 0.1167 - val_accuracy: 0.9503 - val_loss: 0.1270
Epoch 6/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 230s 355ms/step - accuracy: 0.9597 - loss: 0.1188 - val_accuracy: 0.9276 - val_loss: 0.2257
Epoch 7/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 260s 353ms/step - accuracy: 0.9598 - loss: 0.1107 - val_accuracy: 0.9514 - val_loss: 0.1701
Epoch 8/10
646/646 ━━━━━━━━━━━━━━━━━━━━ 255s 342ms/step - accuracy: 0.9645 -

**Evaluate the Model**

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

162/162 ━━━━━━━━━━━━━━━━━━━━ 12s 77ms/step - accuracy: 0.9552 - loss: 0.1363
Test Accuracy: 95.94%


**Save the Model**

In [ ]:
model.save("plascanai.keras")

**Test on New Data**

In [ ]:
# 1. Evaluate on your held‑out test set
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

# 2. Upload a new image file in Colab
from google.colab import files
uploaded = files.upload()  # this opens a file picker in the browser

# 3. Grab the filename of the uploaded file
if uploaded:
    # uploaded is a dict: {filename: file_bytes}
    img_filename = next(iter(uploaded.keys()))

    # 4. Load & preprocess the image
    import cv2
    import numpy as np
    img = cv2.imread(img_filename)
    img = cv2.resize(img, (64, 64))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)

162/162 - 10s - 64ms/step - accuracy: 0.9594 - loss: 0.1251
Test loss: 0.1251, Test accuracy: 0.9594


In [ ]:
# 5. Make prediction
    pred = model.predict(img)[0][0]
    result = "Parasitized (Malaria Detected)" if pred > 0.5 else "Uninfected (No Malaria)"



In [ ]:
  # 6. Display outcome and image
    print(f"Prediction: {result} (confidence: {pred:.3f})")
    import matplotlib.pyplot as plt
    plt.imshow(cv2.cvtColor(cv2.imread(img_filename), cv2.COLOR_BGR2RGB))
    plt.title(result)
    plt.axis('off')
    plt.show()
else:
    print("No file uploaded.")